In [11]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns
import itertools
from sklearn.decomposition import PCA

from sklearn.model_selection             import train_test_split
from recommendation_evaluation_functions import recommend_ingredients, held_out_recommendation, calculate_metrics

np.seterr(divide='ignore', invalid='ignore')
%matplotlib inline

In [2]:
# Load data.
X        = pd.read_csv('train_dataset.csv', sep = '\t', index_col = 0).drop('cuisine', axis = 1)
cuisines = pd.read_csv('train_dataset.csv', sep = '\t', usecols = ['cuisine'])
cuisines.set_index(X.index, inplace = True)

In [20]:
# Apply CF to original recipes.
recommendations = recommend_ingredients(X.head(50), X.head(50), similarity_measure = "jaccard", k = 5)

Some ingredients were not present in any recipe.


In [23]:
# Print some recipe + recommendation samples.
recipe1 = 7
recipe2 = 10

print('Recipe {} ({}): \n{}\n'.format(X.index[recipe1], 
                                      cuisines.iloc[recipe1, 0],
                                      X.columns[np.where(X.iloc[recipe1,] != 0)]))
print('Recommended ingredients: \n{}\n\n'.format(recommendations[5][recipe1,]))

print('Recipe {} ({}): \n{}\n'.format(X.index[recipe2], 
                                      cuisines.iloc[recipe2, 0],
                                      X.columns[np.where(X.iloc[recipe2,] != 0)]))
print('Recommended ingredients: \n{}\n\n'.format(recommendations[5][recipe2,]))

Recipe 16903 (mexican): 
Index(['cheddar_cheese', 'jalapeno_chilies', 'lettuce', 'lime', 'pork',
       'purple_onion', 'peppers', 'olive', 'cilantro_chopped_fresh',
       'pepper_ground_black', 'tortillas_corn'],
      dtype='object')

Recommended ingredients: 
Index(['cilantro_chopped', 'beans', 'olives_black', 'vinegar_white',
       'mushrooms', 'garlic', 'tomatoes', 'chicken_broth', 'sauce_fish',
       'pepper_green'],
      dtype='object')


Recipe 45887 (chinese): 
Index(['canola', 'garlic', 'mustard', 'scallions', 'sesame', 'sugar', 'wine',
       'starch_corn', 'crushed_pepper_flakes', 'ginger_fresh', 'beans_green',
       'sodium_sauce', 'pepper_white'],
      dtype='object')

Recommended ingredients: 
Index(['vegetable', 'pepper', 'paste_curry', 'sesame_seeds', 'olive',
       'carrots', 'ground_pork', 'pepper_green_bell', 'olives', 'capers'],
      dtype='object')




### Evaluation

**Split the data into train and test set**

In [4]:
X_train, X_test = train_test_split(X, test_size = 0.9, random_state = 1)

### Tune alpha parameter cosine similarity

In [8]:
alpha_grid = np.linspace(0.05,0.35,2)

results_alpha_PCA = pd.DataFrame(columns = ["k", "similarity_measure", "top10_presence", "mean_rank", 
                                            "median_rank"])
user_item_matrix  = X_train

# Fix number of neighbours to 50 and use PCA to evaluate (roughly) the optimal value of alpha
model_config = [50, "asymmetric_cosine", len(X.columns)]

for alpha in alpha_grid:   
    
    missing_ingredients, recommendations = held_out_recommendation(user_item_matrix, model_config, usePCA = True,
                                                                   alpha = alpha)
    
    for k, recs in recommendations.items():
        results_alpha_PCA = results_alpha_PCA.append(calculate_metrics(missing_ingredients, recs, model_config[0],  
                                                                       model_config[1]),ignore_index = True)

In [9]:
results_alpha_PCA

,k,similarity_measure,top10_presence,mean_rank,median_rank
0,[50],asymmetric_cosine,0.446438,34.3768,12
1,[50],asymmetric_cosine,0.439743,33.0263,13


In [ ]:
results_alpha_PCA.to_csv('results/parameter_settings_train_alpha.csv', sep = '\t')

### Without PCA

In [16]:
results = pd.DataFrame(columns = ["k", "similarity_measure", "top10_presence", "mean_rank", "median_rank"])#

k_grid              = [5, 10, 20, 50, 100]
similarity_measures = ["cosine", "asymmetric_cosine", "jaccard", "pmi"]
n_recommendations   = len(X.columns)
user_item_matrix    = X_train

for sim in similarity_measures:
    
    model_config = [k_grid, sim, n_recommendations]
    missing_ingredients, recommendations = held_out_recommendation(user_item_matrix, model_config)
    
    for k, recs in recommendations.items():
        results = results.append(calculate_metrics(missing_ingredients, recs, k, sim), 
                                 ignore_index = True)    

In [17]:
results

,k,similarity_measure,top10_presence,mean_rank,median_rank
0,5,cosine,0.255222,71.8642,39


In [ ]:
results.to_csv('results/parameter_settings_train.csv', sep = '\t')

### Same settings, with PCA

In [13]:
results_PCA = pd.DataFrame(columns = ["k", "similarity_measure", "top10_presence", "mean_rank", "median_rank"])

for sim in similarity_measures:
    
    model_config = [k_grid, sim, n_recommendations]
    missing_ingredients, recommendations = held_out_recommendation(user_item_matrix, model_config, usePCA = True)
    
    for k, recs in recommendations.items():
        results_PCA = results_PCA.append(calculate_metrics(missing_ingredients, recs, k, sim), 
                            ignore_index = True)

In [14]:
results_PCA

,k,similarity_measure,top10_presence,mean_rank,median_rank
0,5,cosine,0.33744,68.0973,25
1,10,cosine,0.380557,53.9054,18


In [ ]:
results_PCA.to_csv('results/parameter_settings_train_PCA.csv', sep = '\t')